In [1]:
using StatisticalRethinking, CmdStan, StanMCMCChains
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
#mean_height = mean(df2[:height])
df2[:height_c] = convert(Vector{Float64}, df2[:height]) # .- mean_height
first(df2, 5)

max_height_c = maximum(df2[:height_c])
min_height_c = minimum(df2[:height_c])

heightsmodel = "
data {
  int N;
  real h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=$(min_height_c),upper=$(max_height_c)> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 0.1);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchains);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height_c]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.2s.jls", chn)

chn2 = deserialize("m4.2s.jls")

describe(chn2)


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/heights.stan will be updated.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.031, 0.031, 0.030, 0.030) seconds, 0.12 seconds total
Sampling took (0.046, 0.046, 0.052, 0.049) seconds, 0.19 seconds total

                 Mean     MCSE   StdDev     5%    50%    95%    N_Eff  N_Eff/s    R_hat
lp__            -1301  2.3e-02  9.5e-01  -1303  -1300  -1300  1.7e+03  9.0e+03  1.0e+00
accept_stat__    0.91  1.7e-03  1.2e-01   0.66   0.95    1.0  4.6e+03  2.4e+04  1.0e+00
stepsize__       0.94  6.8e-03  9.6e-03   0.93   0.94   0.95  2.0e+00  1.0e+01  5.9e+12
treedepth__       1.8  7.8e-03  4.8e-01    1.0    2.0    2.0  3.7e+03  1.9e+04  1.0e+00
n_leapfrog__      3.5  4.9e-02  2.2e+00    1.0    3.0    7.0  2.0e+03  1.1e+04  1.0e+00
divergent__      0.00      nan  0.0e+00   0.00   0.00   0.00      nan      nan      nan


end of m4.2s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*